##### Proyecto 2 teoria de la simulacion
Josue Salomon Landa 20211001600 :3

![Hornet](https://images.steamusercontent.com/ugc/1709665624025799474/1D03582C0386590AE8EB073E534CAF0A7A267250/?imw=128&imh=128&ima=fit&impolicy=Letterbox&imcolor=%23000000&letterbox=true)

#### Importacion de librerias

In [678]:
import pandas as pd
import numpy as np
import simpy
import random

#### Creacionde clases para la simulacion

In [679]:
class Mesa:
    def __init__(self, env, id, num_sillas):
        self.env = env
        self.id = id
        self.num_sillas = simpy.Container(env, init=num_sillas, capacity=num_sillas)

In [680]:
class Plato:
    def __init__(self, nombre, tiempo_cocina, tiempo_comida):
        self.nombre = nombre
        self.tiempo_cocina = tiempo_cocina
        self.tiempo_comida = tiempo_comida

In [681]:
class Restaurante:
    def __init__(self, env, mesas, num_meseros, num_chefs, platos):
        self.env = env
        self.mesas = mesas
        self.meseros = simpy.Resource(env, capacity=num_meseros)
        self.chefs = simpy.Resource(env, capacity=num_chefs)
        self.platos = platos


In [682]:
class Cliente:
    def __init__(self, env, id, mesa, restaurante):
        self.env = env
        self.id = id
        self.mesa: Mesa = mesa
        self.restaurante: Restaurante = restaurante
        self.eventos = pd.DataFrame()
        self.grupo_size = random.randint(2, 4) 
        self.alcanzo_disponibilidad = False
        self.plato = None 
        self.tolerancia_chef = random.randint(5, 15)  
        self.env.process(self.start())

    def start(self):
        print(f"🕒 Tiempo: {self.env.now} → Cliente {self.id} llegó al restaurante")
        self.eventos = pd.concat([self.eventos, pd.DataFrame({
            "id": [self.id], "evento": ["LLEGO"], "time": [self.env.now]
        })])

        if self.mesa.num_sillas.level >= self.grupo_size:  
            yield self.mesa.num_sillas.get(self.grupo_size)  
            print(f"🕒 Tiempo: {self.env.now} → Cliente {self.id} se sienta en mesa {self.mesa.id} con {self.grupo_size} personas")
        else:
            print(f"🕒 Tiempo: {self.env.now} → Cliente {self.id} se fue porque no hay suficientes sillas")
            self.eventos = pd.concat([self.eventos, pd.DataFrame({
                "id": [self.id], "evento": ["SIN SILLA"], "time": [self.env.now]
            })])
            return

        with self.restaurante.meseros.request() as req_mesero:
            yield req_mesero
            print(f"🕒 Tiempo: {self.env.now} → Cliente {self.id} está siendo atendido por el mesero")
            self.eventos = pd.concat([self.eventos, pd.DataFrame({
                "id": [self.id], "evento": ["ATENDIDO"], "time": [self.env.now]
            })])

            self.plato = random.choice(self.restaurante.platos)
            print(f"🕒 Tiempo: {self.env.now} → Cliente {self.id} pidió '{self.plato.nombre}' con tolerancia de {self.tolerancia_chef} minutos")

            with self.restaurante.chefs.request() as req_chef:
                resultado = yield req_chef | self.env.timeout(self.tolerancia_chef)

                if req_chef in resultado:
                    print(f"🕒 Tiempo: {self.env.now} → Chef comenzó a cocinar para Cliente {self.id}")
                    yield self.env.timeout(self.plato.tiempo_cocina)
                    print(f"🕒 Tiempo: {self.env.now} → Cliente {self.id} recibió su comida y empezó a comer '{self.plato.nombre}'")

                    # Comer
                    yield self.env.timeout(self.plato.tiempo_comida)
                    print(f"🕒 Tiempo: {self.env.now} → Cliente {self.id} terminó de comer y se fue")

                    self.eventos = pd.concat([self.eventos, pd.DataFrame({
                        "id": [self.id], "evento": ["TERMINO COMIDA"], "time": [self.env.now]
                    })])
                else:
                    print(f"🕒 Tiempo: {self.env.now} → Cliente {self.id} se fue porque el chef tardó demasiado")
                    self.eventos = pd.concat([self.eventos, pd.DataFrame({
                        "id": [self.id], "evento": ["SE FUE POR CHEF"], "time": [self.env.now]
                    })])

        yield self.mesa.num_sillas.put(self.grupo_size)  


In [683]:
class Simulacion:
    def __init__(self, env, mesas_settings, num_meseros, num_chefs, platos_disponibles):
        self.env = env
        self.mesas = [Mesa(env, m["id"], m["num_sillas"]) for m in mesas_settings]
        self.restaurante = Restaurante(env, self.mesas, num_meseros, num_chefs, platos_disponibles)
        self.clientes = []
        self.env.process(self.start())

    def start(self):
        while True:
            yield self.env.timeout(random.randint(1, 4))
            
            mesa_disponible = None
            for mesa in self.mesas:
                if mesa.num_sillas.level >= 2: 
                    mesa_disponible = mesa
                    break
            
            if mesa_disponible: 
                cliente = Cliente(
                    self.env,
                    len(self.clientes) + 1,
                    mesa_disponible,  
                    self.restaurante
                )
                self.clientes.append(cliente)
            else:
                print(f"🕒 Tiempo: {self.env.now} → No hay mesas disponibles para nuevos clientes.")


In [684]:
env = simpy.Environment()

mesas_settings = [
    {"id": 1, "num_sillas": 4},
    {"id": 2, "num_sillas": 4},
    {"id": 3, "num_sillas": 6}
]

platos_disponibles = [
    Plato("Spaghetti", tiempo_cocina=7, tiempo_comida=6),
    Plato("Pizza", tiempo_cocina=10, tiempo_comida=8),
    Plato("Hamburguesa", tiempo_cocina=5, tiempo_comida=4),
    Plato("Ensalada", tiempo_cocina=3, tiempo_comida=9)
]

simulacion = Simulacion(
    env,
    mesas_settings,
    num_meseros=4,
    num_chefs=4,
    platos_disponibles=platos_disponibles
)
random.seed=45
env.run(until=60*5)

🕒 Tiempo: 1 → Cliente 1 llegó al restaurante
🕒 Tiempo: 1 → Cliente 1 se sienta en mesa 1 con 4 personas
🕒 Tiempo: 1 → Cliente 1 está siendo atendido por el mesero
🕒 Tiempo: 1 → Cliente 1 pidió 'Pizza' con tolerancia de 14 minutos
🕒 Tiempo: 1 → Chef comenzó a cocinar para Cliente 1
🕒 Tiempo: 5 → Cliente 2 llegó al restaurante
🕒 Tiempo: 5 → Cliente 2 se sienta en mesa 2 con 2 personas
🕒 Tiempo: 5 → Cliente 2 está siendo atendido por el mesero
🕒 Tiempo: 5 → Cliente 2 pidió 'Spaghetti' con tolerancia de 10 minutos
🕒 Tiempo: 5 → Chef comenzó a cocinar para Cliente 2
🕒 Tiempo: 8 → Cliente 3 llegó al restaurante
🕒 Tiempo: 8 → Cliente 3 se fue porque no hay suficientes sillas
🕒 Tiempo: 9 → Cliente 4 llegó al restaurante
🕒 Tiempo: 9 → Cliente 4 se fue porque no hay suficientes sillas
🕒 Tiempo: 11 → Cliente 1 recibió su comida y empezó a comer 'Pizza'
🕒 Tiempo: 12 → Cliente 2 recibió su comida y empezó a comer 'Spaghetti'
🕒 Tiempo: 13 → Cliente 5 llegó al restaurante
🕒 Tiempo: 13 → Cliente 5 se 

In [685]:
lista_clientes = []

for c in simulacion.clientes:
    lista_clientes.append({
        "id": c.id,
        "grupo_size": c.grupo_size,
        "mesa_id": c.mesa.id if hasattr(c, 'mesa') and c.mesa else None, 
        "plato_seleccionado": c.plato.nombre if c.plato else None, 
        "tiempo_comida": c.plato.tiempo_comida if c.plato else None,  
        "tolerancia_espera_chef": c.tolerancia_chef 
    })

df_clientes = pd.DataFrame(lista_clientes)


In [686]:
df_clientes

,id,grupo_size,mesa_id,plato_seleccionado,tiempo_comida,tolerancia_espera_chef
0,1,4,1,Pizza,8.0,14
1,2,2,2,Spaghetti,6.0,10
2,3,3,2,None,NaN,9
3,4,4,2,None,NaN,12
4,5,4,2,None,NaN,7
...,...,...,...,...,...,...
101,102,3,1,Hamburguesa,4.0,6
102,103,2,3,Pizza,8.0,10
103,104,3,1,Spaghetti,6.0,5
104,105,3,2,Spaghetti,6.0,15


In [687]:
lista_eventos = []

for c in simulacion.clientes:
    lista_eventos.append(c.eventos)

df_eventos = pd.concat(lista_eventos).reset_index(drop=True)

In [688]:
df_eventos

,id,evento,time
0,1,LLEGO,1
1,1,ATENDIDO,1
2,1,TERMINO COMIDA,19
3,2,LLEGO,5
4,2,ATENDIDO,5
...,...,...,...
270,104,ATENDIDO,289
271,105,LLEGO,292
272,105,ATENDIDO,292
273,106,LLEGO,293


#### Preguntas

1) ¿Cuánto tiempo de espera tuvieron los clientes antes de sentarse en una mesa?

In [689]:
llegadas = df_eventos[df_eventos['evento'] == 'LLEGO'].set_index('id')
atendidos = df_eventos[df_eventos['evento'] == 'ATENDIDO'].set_index('id')

tiempos_espera = pd.DataFrame()
tiempos_espera['tiempo_llegada'] = llegadas['time']
tiempos_espera['tiempo_atendido'] = atendidos['time']
tiempos_espera['espera_para_sentarse'] = tiempos_espera['tiempo_atendido'] - tiempos_espera['tiempo_llegada']

promedio_espera = tiempos_espera['espera_para_sentarse'].mean()
max_espera = tiempos_espera['espera_para_sentarse'].max()
min_espera = tiempos_espera['espera_para_sentarse'].min()

print(f"Tiempo promedio de espera para sentarse: {promedio_espera:.2f} minutos")
print(f"Tiempo máximo de espera: {max_espera:.2f} minutos")
print(f"Tiempo mínimo de espera: {min_espera:.2f} minutos")

Tiempo promedio de espera para sentarse: 0.33 minutos
Tiempo máximo de espera: 7.00 minutos
Tiempo mínimo de espera: 0.00 minutos


2. ¿Qué plato fue el más solicitado y cuánto tiempo promedio pasaron los clientes comiendo?

In [690]:
platos_count = df_clientes['plato_seleccionado'].value_counts()
tiempo_promedio_por_plato = df_clientes.groupby('plato_seleccionado')['tiempo_comida'].mean()

plato_mas_popular = platos_count.idxmax()
cantidad_plato_popular = platos_count.max()

print(f"El plato más solicitado fue: {plato_mas_popular} con {cantidad_plato_popular} pedidos")
print("\nTiempo promedio de comida por plato:")
for plato, tiempo in tiempo_promedio_por_plato.items():
    print(f"{plato}: {tiempo:.2f} minutos")

El plato más solicitado fue: Hamburguesa con 21 pedidos

Tiempo promedio de comida por plato:
Ensalada: 9.00 minutos
Hamburguesa: 4.00 minutos
Pizza: 8.00 minutos
Spaghetti: 6.00 minutos


3) ¿Cuántos clientes completaron su comida 

In [691]:
terminaron = df_eventos[df_eventos['evento'] == 'TERMINO COMIDA']['id'].nunique()
total_clientes = len(df_clientes)

porcentaje_terminaron = (terminaron / total_clientes) * 100


print(f"Clientes que completaron su comida: {terminaron} ({porcentaje_terminaron:.1f}%)")
print(f"Total de clientes: {total_clientes}")


tolerancia_terminaron = df_clientes[df_clientes['id'].isin(
    df_eventos[df_eventos['evento'] == 'TERMINO COMIDA']['id'])]['tolerancia_espera_chef'].mean()


print(f"\nTolerancia promedio de quienes terminaron: {tolerancia_terminaron:.2f} minutos")


Clientes que completaron su comida: 63 (59.4%)
Total de clientes: 106

Tolerancia promedio de quienes terminaron: 10.95 minutos


4) ¿En promedio, desde que llegó hasta que se fue, cuánto tiempo invirtió cada cliente en el restaurante?

In [692]:
tiempos = []

for id in df_eventos['id'].unique():
    eventos = df_eventos[df_eventos['id'] == id]
    
    if 'LLEGO' in eventos['evento'].values:
        inicio = eventos[eventos['evento'] == 'LLEGO']['time'].values[0]
        
        for fin_evento in ['TERMINO COMIDA', 'SE FUE POR CHEF', 'SIN SILLA']:
            if fin_evento in eventos['evento'].values:
                fin = eventos[eventos['evento'] == fin_evento]['time'].values[0]
                tiempos.append(fin - inicio)
                break

promedio = sum(tiempos) / len(tiempos) if tiempos else 0
print(f"Tiempo promedio de estancia: {promedio:.2f} minutos")

Tiempo promedio de estancia: 7.77 minutos


5) ¿Cuántos clientes no alcanzaron a ser atendidos?

In [693]:
total_clientes = len(df_clientes)
sin_silla = df_eventos[df_eventos['evento'] == 'SIN SILLA']['id'].nunique()
sin_chef = df_eventos[df_eventos['evento'] == 'SE FUE POR CHEF']['id'].nunique()
total_no_atendidos = sin_silla + sin_chef

print(f"Clientes que no encontraron mesa disponible: {sin_silla} ({sin_silla/total_clientes*100:.2f}%)")
print(f"Total clientes no atendidos completamente: {total_no_atendidos} ({total_no_atendidos/total_clientes*100:.2f}%)")

# Contar clientes que completaron su comida
completaron = df_eventos[df_eventos['evento'] == 'TERMINO COMIDA']['id'].nunique()
print(f"Clientes que completaron su comida: {completaron} ({completaron/total_clientes*100:.2f}%)")

Clientes que no encontraron mesa disponible: 39 (36.79%)
Total clientes no atendidos completamente: 39 (36.79%)
Clientes que completaron su comida: 63 (59.43%)
